In [1]:
import pandas as pd

## UBE_DADOS_DEFASADOS.xls
Contains values with diferent lags sizes for each of the attributes: ["ATP","RBG","BLS","SFB","FZB","UBE"]

## Dados UBE.csv
Contains only the main values. Should use this to build a new input set with the desired lags

In [124]:
dataset_path='Data/Dados UBE.csv'
df = pd.read_csv(dataset_path)
df.head()

,Date,ATP,RBG,BLS,SFB,FZB,UBE
0,02/03/1973,250,244,91,214,37,575
1,03/03/1973,191,305,97,208,36,643
2,04/03/1973,169,380,115,207,36,727
3,05/03/1973,136,290,114,201,38,615
4,06/03/1973,131,286,100,196,38,605


In [125]:
number_of_lags = 8
def create_lagged_dataset(df, number_of_lags=8):
    colunms_shifted = []
    for colunm in df:
        if colunm == "Date":
#             colunms_shifted += [df[colunm]]
            continue

        colunm_data = df[colunm]
        colunms_shifted += [colunm_data.shift(i).rename(colunm_data.name + "_" + str(i)) for i in range(number_of_lags)] 

    lagged_colunms = pd.concat(colunms_shifted, axis=1, sort=False)
    return lagged_colunms

new_df = create_lagged_dataset(df, number_of_lags)
new_df.dropna(inplace=True)

In [126]:
# Get results
results_df = new_df[["ATP_0", "RBG_0", "BLS_0", "SFB_0", "FZB_0", "UBE_0"]]

# Drop results from new_df
df_without_answers = new_df.drop(["ATP_0", "RBG_0", "BLS_0", "SFB_0", "FZB_0", "UBE_0"], axis=1)

# Normalize
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

cols = results_df.columns
results_df_scaled = min_max_scaler.fit_transform(results_df)
results_df_normalized = pd.DataFrame(results_df_scaled, columns=cols)


cols = df_without_answers.columns
df_without_answers_scaled = min_max_scaler.fit_transform(df_without_answers)
df_without_answers_normalized = pd.DataFrame(df_without_answers_scaled, columns=cols)

In [128]:
results_df_normalized.columns


Index(['ATP_0', 'RBG_0', 'BLS_0', 'SFB_0', 'FZB_0', 'UBE_0'], dtype='object')

In [129]:
# Creates train, test, and validation data
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(
    df_without_answers.values, 
    results_df.values,
    test_size=0.25
)

# Reserve a part of the samples for validation
percentage_n = percentage(20, len(train_x))
percentage_n = int(percentage_n)
val_x = train_x[-percentage_n:]
val_y = train_y[-percentage_n:]
train_x = train_x[:-percentage_n]
train_y = train_y[:-percentage_n]

train_set = (train_x, train_y)
test_set = (test_x, test_y)
validation_set = (val_x, val_y)

In [142]:
model = create_network()
print('# Fit model on training data')
history = model.fit(train_set[0], train_set[1],
                    batch_size=64,
                    epochs=50,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=validation_set)

# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(test_set[0], test_set[1], batch_size=64)
print(f"{model.metrics_names}, {results}")

# Fit model on training data
Train on 10241 samples, validate on 2560 samples
Epoch 1/50
10241/10241 [==============================] - 0s 24us/step - loss: 11815.2250 - mean_absolute_percentage_error: 19.2170 - MSE: 5268.7056 - val_loss: 5879.8057 - val_mean_absolute_percentage_error: 12.4045 - val_MSE: 2132.9082
Epoch 2/50
10241/10241 [==============================] - 0s 17us/step - loss: 5056.0735 - mean_absolute_percentage_error: 12.0337 - MSE: 1853.9551 - val_loss: 4382.5703 - val_mean_absolute_percentage_error: 13.0524 - val_MSE: 1921.4827
Epoch 3/50
10241/10241 [==============================] - 0s 17us/step - loss: 3961.8337 - mean_absolute_percentage_error: 11.5992 - MSE: 1705.5264 - val_loss: 3739.0243 - val_mean_absolute_percentage_error: 10.9273 - val_MSE: 1872.4658
Epoch 4/50
10241/10241 [==============================] - 0s 17us/step - loss: 3455.6497 - mean_absolute_percentage_error: 10.7879 - MSE: 1583.9504 - val_loss: 3295.9721 - val_mean_absolute_percentage_error: 9.

10241/10241 [==============================] - 0s 17us/step - loss: 2436.0213 - mean_absolute_percentage_error: 8.8678 - MSE: 1274.8527 - val_loss: 2541.6100 - val_mean_absolute_percentage_error: 9.9701 - val_MSE: 1444.7045
Epoch 36/50
10241/10241 [==============================] - 0s 16us/step - loss: 2438.6917 - mean_absolute_percentage_error: 8.8937 - MSE: 1282.7407 - val_loss: 2474.1490 - val_mean_absolute_percentage_error: 8.3486 - val_MSE: 1362.0986
Epoch 37/50
10241/10241 [==============================] - 0s 17us/step - loss: 2423.3022 - mean_absolute_percentage_error: 8.6394 - MSE: 1269.9216 - val_loss: 2589.0663 - val_mean_absolute_percentage_error: 9.5897 - val_MSE: 1500.4177
Epoch 38/50
10241/10241 [==============================] - 0s 16us/step - loss: 2422.5570 - mean_absolute_percentage_error: 8.8576 - MSE: 1276.9546 - val_loss: 2501.0908 - val_mean_absolute_percentage_error: 8.6469 - val_MSE: 1412.1857
Epoch 39/50
10241/10241 [==============================] - 0s 17us/s

In [139]:
model.metrics_names

['loss', 'mean_absolute_percentage_error', 'MSE']

In [136]:
from keras.regularizers import l2, l1
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


def create_network(input_size=6*(number_of_lags-1), hidden_layer_neurons=480, output_size=6, optimizer="adam",
                   activation_1_layer="relu", activation_2_layer="linear", regulazier_1_layer=l2(0.01),
                   regulazier_2_layer=l2(0.01)):
    classifier = Sequential()

    #camada escondida
    classifier.add(
        Dense(
            units=hidden_layer_neurons,
            activation=activation_1_layer,
            input_dim= input_size,
            kernel_regularizer=regulazier_1_layer,
            activity_regularizer=l1(0.01)
        )
    )

    #camada de saida
    classifier.add(
        Dense(
            units=output_size,
            activation=activation_2_layer,
            kernel_regularizer=regulazier_2_layer,
            activity_regularizer=l1(0.01)
        )
    )

    classifier.compile(optimizer=optimizer, loss='MSE', metrics=['mean_absolute_percentage_error','MSE'])

    return classifier